```
From: https://github.com/ksatola
Version: 0.0.1

TODOs
1. 

```

# Exploring Data (EDA)
By understanding the business and the data, you are able to create better models and have a better impact on their business. EDA gives us a feel for the data, but also is a great excuse to meet and discuss issues with business units that control that data.

In [ ]:
# Connect with underlying Python code
%load_ext autoreload
%autoreload 2
import sys
sys.path.insert(0, '../src')

In [ ]:
from datasets import (
    get_dataset
)

In [ ]:
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
df = get_dataset('titanic3')